In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
resources=pd.read_csv("../Datasets/Resources.csv")
teachers=pd.read_csv("../Datasets/teachers.csv")
courses=pd.read_csv("../Datasets/courses.csv")

In [3]:
resources

,Type_ressource,Identifiant_ressource,Nom_matiere,Filiere,Niveau,Cursus,Annee_academique,Date,Heure_debut,Heure_fin,Type_cours,Periode_annee,Nb_personnes,Capacite,Enseignant_id,Videoprojecteur,Chevauchement,Etat_utilisation_ressource
0,Salle,26BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-09-19,07:30,11:30,CM,Semestre 1,73,120,ENS-GPR-0004,OUI,NON,BIEN_UTILISE
1,Salle,31BS2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-21,12:30,16:30,CM,Semestre 1,73,120,ENS-GPR-0008,OUI,NON,BIEN_UTILISE
2,Salle,27BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-19,12:30,16:30,CM,Semestre 1,73,120,ENS-GPR-0011,OUI,NON,BIEN_UTILISE
3,Salle,22BS1,Chimie Organique,GPR,5,ING,2020/2021,2020-11-02,07:30,11:30,TD,Semestre 1,36,60,ENS-GPR-0006,OUI,NON,BIEN_UTILISE
4,Salle,23BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-09,12:30,16:30,TD,Semestre 1,36,60,ENS-GPR-0005,NON,NON,BIEN_UTILISE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,Salle,26BS1,Structures Métalliques,GCI,4,ING,2022/2023,2023-05-17,07:30,11:30,CM,Semestre 2,7,60,ENS-GCI-0011,NON,NON,SOUS_UTILISE
8747,Salle,18BS2,Structures Métalliques,GCI,4,ING,2022/2023,2023-02-17,07:30,11:30,TD,Semestre 2,3,60,ENS-GCI-0002,NON,NON,BIEN_UTILISE
8748,Salle,25BP1,Structures Métalliques,GCI,4,ING,2022/2023,2023-05-01,07:30,11:30,TD,Semestre 2,3,60,ENS-GCI-0015,NON,NON,BIEN_UTILISE
8749,Labo,Labo-GCI,Structures Métalliques,GCI,4,ING,2022/2023,2023-03-23,07:30,11:30,TP,Semestre 2,2,35,ENS-GCI-0012,NON,NON,BIEN_UTILISE


In [4]:
# Construction de la target et Feature enginiering

besoin_projecteur=[]
for i in range(resources.shape[0]):
    if resources.loc[i,"Type_cours"]=="CM":
        besoin_projecteur.append("OUI")
    if  resources.loc[i,"Type_cours"] =="TD":
        if resources.loc[i,"Filiere"] in ['GIT', 'SDIA', 'GESI', 'EEAT', 'Météorologie']:
            besoin_projecteur.append("OUI")
        else:
            besoin_projecteur.append("NON")
    if resources.loc[i,"Type_cours"] =="TP":
        if resources.loc[i,"Filiere"] in ['GIT', 'SDIA', 'GESI']:
            besoin_projecteur.append("OUI")
        else:
            besoin_projecteur.append("NON")
            
resources["besoin_projecteur"]=besoin_projecteur

resources["taux_occupation"]=resources.loc[:,"Nb_personnes"]/resources.loc[:,"Capacite"]

adequation_equipement=[]
for i in range(resources.shape[0]):
    if (resources.loc[i,"Videoprojecteur"]=="OUI" and resources.loc[i,"besoin_projecteur"]=="OUI") or (resources.loc[i,"Videoprojecteur"]=="NON" and resources.loc[i,"besoin_projecteur"]=="NON"):
        adequation_equipement.append(1)
    if resources.loc[i,"Videoprojecteur"]=="OUI" and resources.loc[i,"besoin_projecteur"]=="NON":
        adequation_equipement.append(0.5)
    if resources.loc[i,"Videoprojecteur"]=="NON" and resources.loc[i,"besoin_projecteur"]=="OUI":
        adequation_equipement.append(0)

resources["adequation_equipement"]=adequation_equipement

adequation_salle=[]
for i in range(resources.shape[0]):
    type_cours = resources.loc[i, "Type_cours"]
    type_ressource = resources.loc[i, "Type_ressource"]

    if (
        (type_cours == "CM" and type_ressource == "Salle") or
        (type_cours == "TD" and type_ressource == "Salle") or
        (type_cours == "TP" and type_ressource == "Labo")
    ):
        adequation_salle.append(1)
    else:
        adequation_salle.append(0)

resources["adequation_salle"]=adequation_salle

score=resources.loc[:,"taux_occupation"]*0.4+resources.loc[:,"adequation_equipement"]*0.3+resources.loc[:,"adequation_salle"]*0.3

resources["Score"]=score

resources.drop(columns=["Chevauchement","Etat_utilisation_ressource"],inplace=True)

End_Type_ressource=[]
for i in resources.loc[:,"Type_ressource"]:
    if i=="Labo":
        End_Type_ressource.append(2)
    else:
        if i=="Salle":
            End_Type_ressource.append(1)
        else:
            End_Type_ressource.append(0)

resources["End_Type_ressource"]=End_Type_ressource

End_Type_cours=[]
for i in resources.loc[:,"Type_cours"]:
    if i=="CM":
        End_Type_cours.append(2)
    else:
        if i=="TD":
            End_Type_cours.append(1)
        else:
            End_Type_cours.append(0)

resources["End_Type_cours"]=End_Type_cours

End_Videoprojecteur=[]
for i in resources.loc[:,"Videoprojecteur"]:
    if i=="OUI":
        End_Videoprojecteur.append(1)
    else:
        End_Videoprojecteur.append(0)

End_besoin_projecteur=[]
for i in resources.loc[:,"besoin_projecteur"]:
    if i=="OUI":
        End_besoin_projecteur.append(1)
    else:
        End_besoin_projecteur.append(0)

resources["End_Videoprojecteur"]=End_Videoprojecteur
resources["End_besoin_projecteur"]=End_besoin_projecteur



In [5]:
resources

,Type_ressource,Identifiant_ressource,Nom_matiere,Filiere,Niveau,Cursus,Annee_academique,Date,Heure_debut,Heure_fin,...,Videoprojecteur,besoin_projecteur,taux_occupation,adequation_equipement,adequation_salle,Score,End_Type_ressource,End_Type_cours,End_Videoprojecteur,End_besoin_projecteur
0,Salle,26BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-09-19,07:30,11:30,...,OUI,OUI,0.608333,1.0,1,0.843333,1,2,1,1
1,Salle,31BS2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-21,12:30,16:30,...,OUI,OUI,0.608333,1.0,1,0.843333,1,2,1,1
2,Salle,27BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-19,12:30,16:30,...,OUI,OUI,0.608333,1.0,1,0.843333,1,2,1,1
3,Salle,22BS1,Chimie Organique,GPR,5,ING,2020/2021,2020-11-02,07:30,11:30,...,OUI,NON,0.600000,0.5,1,0.690000,1,1,1,0
4,Salle,23BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-09,12:30,16:30,...,NON,NON,0.600000,1.0,1,0.840000,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,Salle,26BS1,Structures Métalliques,GCI,4,ING,2022/2023,2023-05-17,07:30,11:30,...,NON,OUI,0.116667,0.0,1,0.346667,1,2,0,1
8747,Salle,18BS2,Structures Métalliques,GCI,4,ING,2022/2023,2023-02-17,07:30,11:30,...,NON,NON,0.050000,1.0,1,0.620000,1,1,0,0
8748,Salle,25BP1,Structures Métalliques,GCI,4,ING,2022/2023,2023-05-01,07:30,11:30,...,NON,NON,0.050000,1.0,1,0.620000,1,1,0,0
8749,Labo,Labo-GCI,Structures Métalliques,GCI,4,ING,2022/2023,2023-03-23,07:30,11:30,...,NON,NON,0.057143,1.0,1,0.622857,2,0,0,0


In [6]:
resources.to_csv("../Datasets/Resources_cleaned.csv",index=False)

In [7]:
import sys
import os
sys.path.insert(0,"../Models/optimisation des ressources")


In [8]:
from RandomForestRegressor import RandomForestRegressor
from DecisionTreeRegressor import decisionTreeRegressor
from GradientBoostingRegressor import GradientBoostingRegressor
from XGBoostRegressor import XGBoostRegressor

In [9]:
model1=RandomForestRegressor(n_trees=100)
model2=decisionTreeRegressor()
model3=GradientBoostingRegressor(learning_rate=1)
model4=XGBoostRegressor(learning_rate=1)

In [10]:
X=resources.loc[:,["Nb_personnes","Capacite","End_Type_ressource","End_Type_cours","End_Videoprojecteur","End_besoin_projecteur"]]
y=resources.loc[:,"Score"]
indices = np.random.permutation(len(resources))
split = int(0.8 * len(resources))

train_idx = indices[:split]
test_idx  = indices[split:]

X_train = X.iloc[train_idx]
X_test  = X.iloc[test_idx]

y_train = y[train_idx]
y_test  = y[test_idx]

In [11]:
model1.fit(X_train.values,y_train.values)
model1.score(X_train.values,y_train.values)

0.9264907906591615

In [12]:
model2.fit(X_train.values,y_train.values)
model2.score(X_train.values,y_train.values)

0.9991960769883699

In [13]:
model3.fit(X_train.values,y_train.values)
model3.score(X_train.values,y_train.values)

0.9998121747957677

In [14]:
model4.fit(X_train.values,y_train.values)
model4.score(X_train.values,y_train.values)

-2.220446049250313e-16

In [15]:
print(model1.predict(X_test.values))
model1.score(X_test.values,y_test.values)

[0.40977967 0.69539435 0.84667219 ... 0.70080353 0.74852303 0.64974406]


0.9241705652108437

In [16]:
print(model2.predict(X_test.values))
model2.score(X_test.values,y_test.values)

[0.36680942 0.62826259 0.9825     ... 0.62826259 0.714      0.63472678]


0.9989482617502874

In [17]:
model3.predict(X_test.values)
model3.score(X_test.values,y_test.values)

0.9997642516819268

In [18]:
print(model4.predict(X_train.values))
model4.score(X_test.values,y_test.values)

[0.66709812 0.66709812 0.66709812 ... 0.66709812 0.66709812 0.66709812]


-0.00013424822181340446

In [19]:
model2.save("decision_tree_class_model.json")


modèle sauvegardé en JSON: decision_tree_class_model.json


In [20]:
resources

,Type_ressource,Identifiant_ressource,Nom_matiere,Filiere,Niveau,Cursus,Annee_academique,Date,Heure_debut,Heure_fin,...,Videoprojecteur,besoin_projecteur,taux_occupation,adequation_equipement,adequation_salle,Score,End_Type_ressource,End_Type_cours,End_Videoprojecteur,End_besoin_projecteur
0,Salle,26BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-09-19,07:30,11:30,...,OUI,OUI,0.608333,1.0,1,0.843333,1,2,1,1
1,Salle,31BS2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-21,12:30,16:30,...,OUI,OUI,0.608333,1.0,1,0.843333,1,2,1,1
2,Salle,27BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-19,12:30,16:30,...,OUI,OUI,0.608333,1.0,1,0.843333,1,2,1,1
3,Salle,22BS1,Chimie Organique,GPR,5,ING,2020/2021,2020-11-02,07:30,11:30,...,OUI,NON,0.600000,0.5,1,0.690000,1,1,1,0
4,Salle,23BP2,Chimie Organique,GPR,5,ING,2020/2021,2020-11-09,12:30,16:30,...,NON,NON,0.600000,1.0,1,0.840000,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,Salle,26BS1,Structures Métalliques,GCI,4,ING,2022/2023,2023-05-17,07:30,11:30,...,NON,OUI,0.116667,0.0,1,0.346667,1,2,0,1
8747,Salle,18BS2,Structures Métalliques,GCI,4,ING,2022/2023,2023-02-17,07:30,11:30,...,NON,NON,0.050000,1.0,1,0.620000,1,1,0,0
8748,Salle,25BP1,Structures Métalliques,GCI,4,ING,2022/2023,2023-05-01,07:30,11:30,...,NON,NON,0.050000,1.0,1,0.620000,1,1,0,0
8749,Labo,Labo-GCI,Structures Métalliques,GCI,4,ING,2022/2023,2023-03-23,07:30,11:30,...,NON,NON,0.057143,1.0,1,0.622857,2,0,0,0


In [31]:
courses

,nom_cours,level,specialite,CM,TD,TP,total,filiere,departement
0,Mathématiques Générales I,1,Mathématiques,40,30,10,80,Tronc Commun,Tronc Commun
1,Physique Générale I,1,Physique,40,20,10,70,Tronc Commun,Tronc Commun
2,Chimie Générale,1,Chimie,30,15,15,60,Tronc Commun,Tronc Commun
3,Informatique de base,1,Informatique,20,20,20,60,Tronc Commun,Tronc Commun
4,Algèbre Linéaire,1,Mathématiques,35,25,0,60,Tronc Commun,Tronc Commun
...,...,...,...,...,...,...,...,...,...
395,Commande optimale,5,Systèmes,28,20,12,60,EEAT,Département Sciences et Ingénierie (SCI ING)
396,Télécommunications spatiales,5,Télécommunications,24,16,12,52,EEAT,Département Sciences et Ingénierie (SCI ING)
397,Réseaux mobiles 5G,5,Télécommunications,28,16,16,60,EEAT,Département Sciences et Ingénierie (SCI ING)
398,Compatibilité électromagnétique,5,Électronique,24,16,12,52,EEAT,Département Sciences et Ingénierie (SCI ING)


In [22]:
teachers

,Matricule_enseignant,Departement,Specialite,Niveau_academique,Type_enseignant,Anciennete,Score_appreciation,Disponibilite_hebdomadaire,Heures_totales_assignees,Heures_restantes
0,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19
1,ENS-GIT-0002,GIT,Informatique,DOCTEUR,PERMANENT,12,3.40,27,178,0
2,ENS-GIT-0003,GIT,Systèmes,INGENIEUR,PERMANENT,11,3.10,20,155,20
3,ENS-GIT-0004,GIT,Systèmes,INGENIEUR,PERMANENT,12,3.58,29,92,83
4,ENS-GIT-0005,GIT,Réseaux,DOCTEUR,VACATAIRE,10,3.51,8,75,25
...,...,...,...,...,...,...,...,...,...,...
120,ENS-TELECOM-0006,TELECOM,Télécommunications,DOCTEUR,PERMANENT,9,3.24,25,116,59
121,ENS-TELECOM-0007,TELECOM,Électronique,DOCTEUR,VACATAIRE,15,3.33,13,77,23
122,ENS-TELECOM-0008,TELECOM,Électronique,INGENIEUR,PERMANENT,6,2.95,21,116,59
123,ENS-TELECOM-0009,TELECOM,Télécommunications,INGENIEUR,PERMANENT,11,3.22,25,120,55


In [25]:
resources.columns

Index(['Type_ressource', 'Identifiant_ressource', 'Nom_matiere', 'Filiere',
       'Niveau', 'Cursus', 'Annee_academique', 'Date', 'Heure_debut',
       'Heure_fin', 'Type_cours', 'Periode_annee', 'Nb_personnes', 'Capacite',
       'Enseignant_id', 'Videoprojecteur', 'besoin_projecteur',
       'taux_occupation', 'adequation_equipement', 'adequation_salle', 'Score',
       'End_Type_ressource', 'End_Type_cours', 'End_Videoprojecteur',
       'End_besoin_projecteur'],
      dtype='object')

In [26]:
teachers.columns

Index(['Matricule_enseignant', 'Departement', 'Specialite',
       'Niveau_academique', 'Type_enseignant', 'Anciennete',
       'Score_appreciation', 'Disponibilite_hebdomadaire',
       'Heures_totales_assignees', 'Heures_restantes'],
      dtype='object')

In [27]:
teachers.rename(columns={"Matricule_enseignant":"Enseignant_id"},inplace=True)

In [28]:
teachers_resources=pd.merge(teachers,resources,left_on="Enseignant_id",right_on="Enseignant_id",how="inner")

In [29]:
teachers_resources

,Enseignant_id,Departement,Specialite,Niveau_academique,Type_enseignant,Anciennete,Score_appreciation,Disponibilite_hebdomadaire,Heures_totales_assignees,Heures_restantes,...,Videoprojecteur,besoin_projecteur,taux_occupation,adequation_equipement,adequation_salle,Score,End_Type_ressource,End_Type_cours,End_Videoprojecteur,End_besoin_projecteur
0,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,NON,OUI,0.116667,0.0,1,0.346667,1,1,0,1
1,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,NON,OUI,0.658333,0.0,1,0.563333,1,2,0,1
2,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,NON,OUI,0.650000,0.0,1,0.560000,1,1,0,1
3,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,OUI,OUI,0.658333,1.0,1,0.863333,1,2,1,1
4,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,NON,OUI,0.083333,0.0,1,0.333333,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,NON,OUI,0.016667,0.0,1,0.306667,1,1,0,1
8747,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,NON,NON,0.025000,1.0,1,0.610000,2,0,0,0
8748,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,NON,OUI,0.050000,0.0,1,0.320000,1,2,0,1
8749,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,NON,NON,0.028571,1.0,1,0.611429,2,0,0,0


In [32]:
teachers_resources.columns

Index(['Enseignant_id', 'Departement', 'Specialite', 'Niveau_academique',
       'Type_enseignant', 'Anciennete', 'Score_appreciation',
       'Disponibilite_hebdomadaire', 'Heures_totales_assignees',
       'Heures_restantes', 'Type_ressource', 'Identifiant_ressource',
       'Nom_matiere', 'Filiere', 'Niveau', 'Cursus', 'Annee_academique',
       'Date', 'Heure_debut', 'Heure_fin', 'Type_cours', 'Periode_annee',
       'Nb_personnes', 'Capacite', 'Videoprojecteur', 'besoin_projecteur',
       'taux_occupation', 'adequation_equipement', 'adequation_salle', 'Score',
       'End_Type_ressource', 'End_Type_cours', 'End_Videoprojecteur',
       'End_besoin_projecteur'],
      dtype='object')

In [33]:
teachers_resources.rename(columns={"Nom_matiere":"nom_cours"},inplace=True)

In [37]:
teachers_resources_courses=pd.merge(teachers_resources,courses,left_on="nom_cours",right_on="nom_cours",how="inner")

In [38]:
teachers_resources_courses

,Enseignant_id,Departement,Specialite,Niveau_academique,Type_enseignant,Anciennete,Score_appreciation,Disponibilite_hebdomadaire,Heures_totales_assignees,Heures_restantes,...,End_Videoprojecteur,End_besoin_projecteur,level,specialite,CM,TD,TP,total,filiere,departement
0,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,0,1,3,Mathématiques,28,20,4,52,SDIA,Département Informatique et Télécommunications
1,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,0,1,3,Mathématiques,28,20,4,52,SDIA,Département Informatique et Télécommunications
2,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,1,1,3,Systèmes,20,16,12,48,GIT,Département Informatique et Télécommunications
3,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,0,1,3,IA,28,16,20,64,SDIA,Département Informatique et Télécommunications
4,ENS-GIT-0001,GIT,Systèmes,INGENIEUR,PERMANENT,7,3.49,30,156,19,...,1,1,4,Informatique,24,12,20,56,SDIA,Département Informatique et Télécommunications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3853,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,1,1,2,Mathématiques,40,30,10,80,Tronc Commun,Tronc Commun
3854,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,1,1,1,Chimie,30,15,15,60,Tronc Commun,Tronc Commun
3855,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,0,0,1,Physique,40,20,10,70,Tronc Commun,Tronc Commun
3856,ENS-TELECOM-0010,TELECOM,Électronique,INGENIEUR,PERMANENT,10,2.97,27,158,17,...,1,1,1,Chimie,30,15,15,60,Tronc Commun,Tronc Commun


In [62]:
def score(df):
    score_niveau = []
    score_heure = []
    score_pse = []
    
    for i in range(df.shape[0]):
        # PSE
        if df.loc[i,"Specialite"] == df.loc[i,"specialite"]:
            score_pse.append(1)
        else:
            score_pse.append(0)
        
        # Heures
        if df.loc[i,"Heures_restantes"] >= df.loc[i,"total"]:
            score_heure.append(1)
        else:
            score_heure.append(0)
        
        # Niveau académique + Type enseignant combinés
        # Ici on peut sommer les points
        niveau_score = 0
        if df.loc[i,"Niveau_academique"] == "INGENIEUR":
            niveau_score += 3
        elif df.loc[i,"Niveau_academique"] == "MASTER":
            niveau_score += 3
        elif df.loc[i,"Niveau_academique"] == "DOCTEUR":  # attention à l'espace
            niveau_score += 4
        else:
            niveau_score += 5

        if df.loc[i,"Type_enseignant"] == "PERMANENT":
            niveau_score += 1
        else:
            niveau_score += 0

        score_niveau.append(niveau_score)
    
    return score_niveau, score_heure, score_pse,niveau_score


teachers_resources_courses["score_niveau"], teachers_resources_courses["score_heure"], teachers_resources_courses["score_pse"], teachers_resources_courses["niveau_score"] = score(teachers_resources_courses)





In [63]:
teachers_resources_courses["Score"]=teachers_resources_courses["score_niveau"]*0.1+teachers_resources_courses["score_heure"]*0.4+teachers_resources_courses["Anciennete"]*0.2+teachers_resources_courses["Score_appreciation"]*0.1+teachers_resources_courses["score_pse"]*0.2

In [64]:
teachers_resources_courses.columns

Index(['Enseignant_id', 'Departement', 'Specialite', 'Niveau_academique',
       'Type_enseignant', 'Anciennete', 'Score_appreciation',
       'Disponibilite_hebdomadaire', 'Heures_totales_assignees',
       'Heures_restantes', 'Type_ressource', 'Identifiant_ressource',
       'nom_cours', 'Filiere', 'Niveau', 'Cursus', 'Annee_academique', 'Date',
       'Heure_debut', 'Heure_fin', 'Type_cours', 'Periode_annee',
       'Nb_personnes', 'Capacite', 'Videoprojecteur', 'besoin_projecteur',
       'taux_occupation', 'adequation_equipement', 'adequation_salle', 'Score',
       'End_Type_ressource', 'End_Type_cours', 'End_Videoprojecteur',
       'End_besoin_projecteur', 'level', 'specialite', 'CM', 'TD', 'TP',
       'total', 'filiere', 'departement', 'score_niveau', 'score_heure',
       'score_pse', 'niveau_score'],
      dtype='object')

In [65]:
X=teachers_resources_courses.loc[:,["Anciennete","Score_appreciation","score_niveau","score_heure","score_pse"]]
y=teachers_resources_courses.loc[:,"Score"]
indices = np.random.permutation(len(teachers_resources_courses))
split = int(0.8 * len(teachers_resources_courses))

train_idx = indices[:split]
test_idx  = indices[split:]

X_train = X.iloc[train_idx]
X_test  = X.iloc[test_idx]

y_train = y[train_idx]
y_test  = y[test_idx]

In [66]:
model5=decisionTreeRegressor()

In [67]:
model5.fit(X_train.values,y_train.values)
model5.score(X_train.values,y_train.values)

0.9999947656659666

In [ ]:
print("Test score:", model5.score(X_test.values,y_test.values))

0.999748963370112

In [77]:
model6=RandomForestRegressor()

In [78]:
model6.fit(X_train.values,y_train.values)
model6.score(X_train.values,y_train.values)

0.9979173046822308

In [79]:
model7=GradientBoostingRegressor()
model7.fit(X_train.values,y_train.values)
model7.score(X_train.values,y_train.values)

0.999989215259275

In [80]:
model7.save("gradient_boosting_regressor_model_teachers.json")